# Seasonality analsis

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import timedelta, datetime
import pm4py

# Import & data preparation

In [14]:
#path = "../data/raw/BPI_2012_A.xes"
#path = "../data/raw/BPI_2012_O.xes"
#path = "../data/raw/BPI_2012_W.xes"
#path = "../data/raw/BPI_2013_closed_problems.xes"
#path = "../data/raw/helpdesk.xes"
#path = "../data/raw/prepaid_travel_cost.xes"
#path = "../data/raw/request_for_payment.xes"
path = "../data/raw/sepsis.xes"

df = pm4py.read_xes(path, return_dataframe=True)
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'])

parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 2028.06it/s]


In [ ]:
# get dataframe for analysis by events
df_events = df[["case:concept:name", "concept:name", "time:timestamp"]].sort_values(by=["case:concept:name", "time:timestamp"])

In [16]:
# get dataframe for analysis by case durations
df_case_durations = (
    df.groupby('case:concept:name')
    .agg(
        start_time=('time:timestamp', 'min'),
        end_time=('time:timestamp', 'max')
    )
    .assign(
        case_duration=lambda x: x['end_time'] - x['start_time'],
        mid_time=lambda x: x['start_time'] + (x['end_time'] - x['start_time']) / 2
    )
    .reset_index()
)

# Optional: sort by end_time if you want the most recent cases last
df_case_durations = df_case_durations.sort_values('end_time')

if "duration_days" not in df_case_durations.columns:
    df_case_durations["case_duration_seconds"] = df_case_durations["case_duration"].dt.total_seconds()

## Rayleigh test

In [17]:
def rayleigh_test(period, times, values=None):
    """
    Weighted Rayleigh test for event times with associated magnitudes.
    times: array of event times (float, e.g., seconds)
    values: array of associated magnitudes (weights)
    period: hypothesized period (same units as times)
    """
    t = np.asarray(times, dtype=float)
    if values is None:
        w = np.ones_like(t, dtype=float)
    else:
        w = np.asarray(values, dtype=float)        

    phi = 2 * np.pi * (t % period) / period

    C = np.sum(w * np.cos(phi))
    S = np.sum(w * np.sin(phi))
    R = np.sqrt(C**2 + S**2)

    Z = (R**2) / np.sum(w**2)
    p = np.exp(-Z)  # approximate significance
    return Z, p


### by events

In [18]:
t = df_events["time:timestamp"].astype("int64")
# convert timestamps (ns) to seconds for use with period in seconds
times_s = t.astype(float) / 1e9

periods = {
    "1 day": 24 * 3600,
    "1 week": 7 * 24 * 3600,
    "1 month": 30.44 * 24 * 3600,   # approximate month = 30 days
    "1 year": 365 * 24 * 3600,   # approximate year = 365 days
}

for name, per_s in periods.items():
    Z, p = rayleigh_test(per_s, times_s)
    
    if p < 0.001:
        sig = '***'
    elif p < 0.01:
        sig = '**'
    elif p < 0.05:
        sig = '*'
    else:
        sig = ''
    print(f"{name}: Z = {Z:.3f}, p = {p:.3f} {sig}")

print("\nSignificance legend:  ***: p < 0.001 | **: p < 0.01 | *: p < 0.05 | (none) : p >= 0.05")


1 day: Z = 1081.998, p = 0.000 ***
1 week: Z = 3.373, p = 0.034 *
1 month: Z = 22.895, p = 0.000 ***
1 year: Z = 51.981, p = 0.000 ***

Significance legend:  ***: p < 0.001 | **: p < 0.01 | *: p < 0.05 | (none) : p >= 0.05


### by case duration

In [19]:
t = df_case_durations["end_time"].astype("int64")

# convert timestamps (ns) to seconds for use with period in seconds
times_s = t.astype(float) / 1e9
values = df_case_durations["case_duration_seconds"].astype(float)

periods = {
    "1 day": 24 * 3600,
    "1 week": 7 * 24 * 3600,
    "1 month": 30.44 * 24 * 3600,   # approximate month = 30 days
    "1 year": 365 * 24 * 3600,   # approximate year = 365 days
}

for name, per_s in periods.items():
    Z, p = rayleigh_test(per_s, times_s, values=values)
    if p < 0.001:
        sig = '***'
    elif p < 0.01:
        sig = '**'
    elif p < 0.05:
        sig = '*'
    else:
        sig = ''
    print(f"{name}: Z = {Z:.3f}, p = {p:.3f} {sig}")

print("\nSignificance legend:  ***: p < 0.001 | **: p < 0.01 | *: p < 0.05 | (none) : p >= 0.05")

1 day: Z = 26.557, p = 0.000 ***
1 week: Z = 2.747, p = 0.064 
1 month: Z = 0.814, p = 0.443 
1 year: Z = 2.874, p = 0.056 

Significance legend:  ***: p < 0.001 | **: p < 0.01 | *: p < 0.05 | (none) : p >= 0.05
